In [1]:
from py2neo import Graph,Node,Relationship
import pandas as pd

In [5]:
graph = Graph("neo4j://localhost:7687", auth=("neo4j", "jj20jj20"))
# 清除图表
graph.run("match (n) detach delete n")

(No data)

In [6]:
# 获取数据
relations = pd.read_csv('song_relations.csv')
relations.columns = ['id', 'h', 'relation', 't']

In [7]:
relations

,id,h,relation,t
0,0,The Show,歌手,林俊杰
1,1,The Show,歌手,Steve Aoki
2,2,The Show,作词,Steve Aoki
3,3,The Show,作词,Carl Ryden
4,4,The Show,作词,James Daniel Lewis
...,...,...,...,...
1180,1180,会有那么一天,作曲,林俊杰
1181,1181,不懂,歌手,林俊杰
1182,1182,不懂,所属专辑,乐行者
1183,1183,不懂,作词,张思尔


In [8]:
# 将人物和歌曲作为实体
humans = {}
songs = {}

for _, row in relations.iterrows():
    if row['relation'] != "所属专辑":
        if row['h'] not in songs:
            songs[row['h']] = Node("歌曲", name=row['h'])
        if row['t'] not in humans:
            humans[row['t']] = Node("人物", name=row['t'])

In [9]:
# 所属专辑作为歌曲属性
for _, row in relations.iterrows():
    if row['relation'] == "所属专辑":
        songs[row['h']]['专辑'] = row['t']

In [10]:
# 进行关系制作
for _, row in relations.iterrows():
    if row['relation'] != "所属专辑":
        r = Relationship(songs[row['h']], row['relation'], humans[row['t']])
        graph.create(r)

In [11]:
graph

Graph('neo4j://localhost:7687')